In [1]:
import os
import PyPDF2

def extract_text_from_pdfs(folder_path):
    pdf_texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_file = os.path.join(folder_path, filename)
            with open(pdf_file, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                pdf_text = ""
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    pdf_text += page.extract_text()
                pdf_texts.append({"filename": filename, "text": pdf_text})
    return pdf_texts

# Usage
folder_path = "./Data"  # Replace with the path where your PDFs are stored
pdf_texts = extract_text_from_pdfs(folder_path)

for pdf in pdf_texts:
    print(f"First 1000 characters of {pdf['filename']}:")
    print(pdf['text'][:1000])  # Print the first 1000 characters for review


First 1000 characters of CNT growth functions.pdf:
Applied  Surface  Science  332  (2015)  756–760
Contents  lists  available  at  ScienceDirect
Applied  Surface  Science
journal  h  om  epa  ge:  www.elsevier.com/locate/apsusc
A  new understanding of  carbon  nanotube  growth:  Different  functions
of
 carbon species
Yueling  Zhanga,∗,  Baojun  Wangb,  Qing  Yuc,  Yajun  Tiand
aCollege  of  Engineering,  Peking  University,  Summer  Palace  Road  5,  Beijing  100871,  PR  China
bKey  Laboratory  of  Coal  Science  and  Technology,  Taiyuan  University  of  Technology,  Yingze  West  Street  79,  Taiyuan  030024,  PR  China
cSchool  of  Chemistry  and  Chemical  Engineer  and  Center  of  Modern  Analysis,  Nanjing  University,  Hankou  Road  22,  Nanjing  210093,  PR  China
dNational  Institute  of  Clean-and-low-carbon  Energy,  P.O.  Box  001  Shenhua  NICE,  Future  Science  &  Technology  Park,  Beijing  102209,  PR  China
a r t  i  c  l  e  i  n  f  o
Article history:
Received
 1

In [2]:
import re

def generate_qa_pairs(pdf_texts):
    qa_pairs = []
    
    for pdf in pdf_texts:
        text = pdf['text']
        
        # Example 1: Temperature ranges for CNT growth
        temp_match = re.search(r'temperature range.*?(\d+°C–\d+°C)', text)
        if temp_match:
            question = "What is the temperature range for CNT growth in the study?"
            answer = f"The temperature range for CNT growth is {temp_match.group(1)}."
            qa_pairs.append({"question": question, "answer": answer})

        # Example 2: CNT growth mechanism
        if "growth mechanism" in text.lower():
            question = "What mechanism was used for carbon nanotube growth?"
            answer = "The CNT growth mechanism involves chemical vapor deposition (CVD) with acetylene and a catalyst."
            qa_pairs.append({"question": question, "answer": answer})

        # Example 3: Characterization methods for CNT
        if "characterized by" in text.lower():
            question = "What techniques were used to characterize CNTs?"
            answer = "The CNTs were characterized by scanning electron microscopy (SEM), transmission electron microscopy (TEM), and Raman spectroscopy."
            qa_pairs.append({"question": question, "answer": answer})

        # Example 4: Impact of catalyst lifetime
        if "catalyst lifetime" in text.lower():
            question = "How does catalyst lifetime affect CNT growth?"
            answer = "Catalyst lifetime affects CNT growth by limiting the time during which the catalyst can facilitate CNT formation before deactivation."
            qa_pairs.append({"question": question, "answer": answer})

        # Add more Q&A patterns based on the structure of your texts
    
    return qa_pairs

# Example usage
qa_pairs = generate_qa_pairs(pdf_texts)

# Output the generated Q&A pairs
for pair in qa_pairs:
    print(f"Q: {pair['question']}")
    print(f"A: {pair['answer']}")
    print()


Q: What mechanism was used for carbon nanotube growth?
A: The CNT growth mechanism involves chemical vapor deposition (CVD) with acetylene and a catalyst.

Q: What techniques were used to characterize CNTs?
A: The CNTs were characterized by scanning electron microscopy (SEM), transmission electron microscopy (TEM), and Raman spectroscopy.

Q: How does catalyst lifetime affect CNT growth?
A: Catalyst lifetime affects CNT growth by limiting the time during which the catalyst can facilitate CNT formation before deactivation.



In [3]:
import pandas as pd
from datasets import Dataset

# Convert Q&A pairs to DataFrame
df = pd.DataFrame(qa_pairs)

# Create a Hugging Face dataset
dataset = Dataset.from_pandas(df)
print(dataset)


c:\Users\VANI SETH\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question', 'answer'],
    num_rows: 3
})


In [4]:
df

,question,answer
0,What mechanism was used for carbon nanotube gr...,The CNT growth mechanism involves chemical vap...
1,What techniques were used to characterize CNTs?,The CNTs were characterized by scanning electr...
2,How does catalyst lifetime affect CNT growth?,Catalyst lifetime affects CNT growth by limiti...


In [5]:
from transformers import BertTokenizer

# Load SciBERT tokenizer
tokenizer = BertTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["question"], examples["answer"], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)


vocab.txt: 100%|██████████| 228k/228k [00:00<00:00, 7.45MB/s]
c:\Users\VANI SETH\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\VANI SETH\.cache\huggingface\hub\models--allenai--scibert_scivocab_uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 10

In [6]:
from transformers import BertForQuestionAnswering, Trainer, TrainingArguments

# Load SciBERT model for question answering
model = BertForQuestionAnswering.from_pretrained("allenai/scibert_scivocab_uncased")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

pytorch_model.bin: 100%|██████████| 442M/442M [00:28<00:00, 15.3MB/s] 
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification 

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [7]:
!pip3 install accelerate transformers[torch] -U


                                              0.0/324.4 kB ? eta -:--:--
     -------------                          112.6/324.4 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 324.4/324.4 kB 4.0 MB/s eta 0:00:00
                                              0.0/9.5 MB ? eta -:--:--
     -                                        0.5/9.5 MB 9.4 MB/s eta 0:00:01
     -----                                    1.2/9.5 MB 11.1 MB/s eta 0:00:01
     ---------                                2.2/9.5 MB 13.8 MB/s eta 0:00:01
     ------------                             3.1/9.5 MB 15.0 MB/s eta 0:00:01
     ----------------                         3.9/9.5 MB 15.6 MB/s eta 0:00:01
     -------------------                      4.6/9.5 MB 15.6 MB/s eta 0:00:01
     -----------------------                  5.5/9.5 MB 15.9 MB/s eta 0:00:01
     -------------------------                6.1/9.5 MB 15.6 MB/s eta 0:00:01
     --------------------------               6.3/9.5 MB 14.4 MB/

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\VANI SETH\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\~afetensors\\_safetensors_rust.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from transformers import BertForQuestionAnswering, Trainer, TrainingArguments, BertTokenizer

# Load SciBERT model for question answering
model = BertForQuestionAnswering.from_pretrained("allenai/scibert_scivocab_uncased")

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["question"], examples["answer"], truncation=True, padding=True)

# Example: Assume tokenized_dataset has been created
# Initialize the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized f

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
sk-IewRtt-d9VEzjxxBg6gvvh5Xziod8p0OfxRGXPqC2ET3BlbkFJMVpirURkiA2w-awN2_AnTPh-rhP_kLMeRMZnysHsIA

In [10]:
import os
import openai
import PyPDF2
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
from tqdm.auto import tqdm

# Set your OpenAI API key
openai.api_key = "sk-IewRtt-d9VEzjxxBg6gvvh5Xziod8p0OfxRGXPqC2ET3BlbkFJMVpirURkiA2w-awN2_AnTPh-rhP_kLMeRMZnysHsIA" 

# Step 1: Extract Text from Multiple PDFs
def extract_text_from_pdfs(folder_path):
    pdf_texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_file = os.path.join(folder_path, filename)
            with open(pdf_file, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                pdf_text = ""
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    pdf_text += page.extract_text()
                pdf_texts.append({"filename": filename, "text": pdf_text})
    return pdf_texts

# Step 2: Use SciBERT for Question Answering
class SciBERTQA:
    def __init__(self, model_name="allenai/scibert_scivocab_uncased"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForQuestionAnswering.from_pretrained(model_name)

    def answer_question(self, question, context):
        inputs = self.tokenizer(question, context, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
            all_tokens = self.tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
            answer = ' '.join(all_tokens[torch.argmax(start_scores): torch.argmax(end_scores)+1])
        return answer.replace(" ##", "")  # Fix tokenization artifacts

# Step 3: Use GPT-3.5-turbo to Generate Fluent Response
def generate_response_with_gpt(question, answer):
    prompt = f"Question: {question}\nAnswer: {answer}\nCan you expand on this and provide a detailed explanation?"
    
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",
        prompt=prompt,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )
    
    return response.choices[0].text.strip()

def answer_question_with_truncation(question, context, model, tokenizer, max_length=512):
    # Tokenize the question and context with truncation
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=max_length, padding=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits
        
        # Get the answer tokens
        all_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
        answer = ' '.join(all_tokens[torch.argmax(start_scores): torch.argmax(end_scores)+1])
        
    return answer.replace(" ##", "")  # Fix tokenization artifacts

# Now use this modified function in your QA pipeline
def qa_pipeline(folder_path):
    # Extract text from PDFs
    pdf_texts = extract_text_from_pdfs(folder_path)

    # Initialize SciBERT QA model
    scibert_qa = SciBERTQA()

    for pdf in pdf_texts:
        print(f"\n--- Processing PDF: {pdf['filename']} ---\n")
        
        # Define your question
        question = "What is the growth temperature for carbon nanotubes?"

        # Get the answer from SciBERT with truncation
        answer = answer_question_with_truncation(question, pdf['text'], scibert_qa.model, scibert_qa.tokenizer)
        print(f"SciBERT Answer: {answer}")

        # Get a detailed response from GPT-3.5-turbo
        detailed_response = generate_response_with_gpt(question, answer)
        print(f"GPT-3.5 Generated Response: {detailed_response}\n")

# Run the pipeline
folder_path = "./Data"  # Replace with the actual folder path containing your PDFs
qa_pipeline(folder_path)


Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized f


--- Processing PDF: CNT growth functions.pdf ---

SciBERT Answer: 


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [11]:
import os
import PyPDF2
import torch
from transformers import BertForQuestionAnswering, BertTokenizer, T5Tokenizer, T5ForConditionalGeneration

# Step 1: Extract Text from PDFs
def extract_text_from_pdfs(folder_path):
    pdf_texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_file = os.path.join(folder_path, filename)
            with open(pdf_file, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                pdf_text = ""
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    pdf_text += page.extract_text()
                pdf_texts.append({"filename": filename, "text": pdf_text})
    return pdf_texts

# Step 2: SciBERT for Question Answering
class SciBERTQA:
    def __init__(self, model_name="allenai/scibert_scivocab_uncased"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForQuestionAnswering.from_pretrained(model_name)

    def answer_question(self, question, context, max_length=512):
        # Tokenize with truncation to avoid long input issues
        inputs = self.tokenizer(question, context, return_tensors="pt", truncation=True, max_length=max_length, padding=True)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
            all_tokens = self.tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
            answer = ' '.join(all_tokens[torch.argmax(start_scores): torch.argmax(end_scores)+1])
        return answer.replace(" ##", "")  # Fix tokenization artifacts

# Step 3: T5 for Generating Fluent Responses
class T5ResponseGenerator:
    def __init__(self, model_name="t5-base"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)

    def generate_response(self, question, answer, max_length=150):
        input_text = f"question: {question} context: {answer}"
        inputs = self.tokenizer.encode(input_text, return_tensors="pt", truncation=True)
        
        outputs = self.model.generate(inputs, max_length=max_length, num_beams=5, early_stopping=True)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 4: Full Pipeline - Combine SciBERT and T5
def qa_pipeline(folder_path):
    # Extract text from PDFs
    pdf_texts = extract_text_from_pdfs(folder_path)

    # Initialize SciBERT QA model and T5 generator
    scibert_qa = SciBERTQA()
    t5_generator = T5ResponseGenerator()

    for pdf in pdf_texts:
        print(f"\n--- Processing PDF: {pdf['filename']} ---\n")
        
        # Define your question (this can be dynamic or user-input)
        question = "What is the growth temperature for carbon nanotubes?"

        # Step 1: Use SciBERT to get a concise answer
        answer = scibert_qa.answer_question(question, pdf['text'])
        print(f"SciBERT Answer: {answer}")

        # Step 2: Use T5 to generate a more detailed response
        detailed_response = t5_generator.generate_response(question, answer)
        print(f"T5 Generated Response: {detailed_response}\n")

# Run the pipeline
folder_path = "./Data"  # Replace with the actual folder path containing your PDFs
qa_pipeline(folder_path)


Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized f

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [12]:
!pip3 install sentencepiece

                                              0.0/991.5 kB ? eta -:--:--
     ---------                              245.8/991.5 kB 7.6 MB/s eta 0:00:01
     ------------------------               634.9/991.5 kB 6.6 MB/s eta 0:00:01
     -------------------------------------  983.0/991.5 kB 8.9 MB/s eta 0:00:01
     -------------------------------------- 991.5/991.5 kB 6.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import os
import PyPDF2
import torch
from transformers import BertForQuestionAnswering, BertTokenizer, T5Tokenizer, T5ForConditionalGeneration

# Step 1: Extract Text from PDFs
def extract_text_from_pdfs(folder_path):
    pdf_texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_file = os.path.join(folder_path, filename)
            with open(pdf_file, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                pdf_text = ""
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    pdf_text += page.extract_text()
                pdf_texts.append({"filename": filename, "text": pdf_text})
    return pdf_texts

# Step 2: SciBERT for Question Answering
class SciBERTQA:
    def __init__(self, model_name="allenai/scibert_scivocab_uncased"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForQuestionAnswering.from_pretrained(model_name)

    def answer_question(self, question, context, max_length=512):
        # Tokenize with truncation to avoid long input issues
        inputs = self.tokenizer(question, context, return_tensors="pt", truncation=True, max_length=max_length, padding=True)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
            all_tokens = self.tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
            answer = ' '.join(all_tokens[torch.argmax(start_scores): torch.argmax(end_scores)+1])
        return answer.replace(" ##", "")  # Fix tokenization artifacts

# Step 3: T5 for Generating Fluent Responses
class T5ResponseGenerator:
    def __init__(self, model_name="t5-base"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)

    def generate_response(self, question, answer, max_length=150):
        input_text = f"question: {question} context: {answer}"
        inputs = self.tokenizer.encode(input_text, return_tensors="pt", truncation=True)
        
        outputs = self.model.generate(inputs, max_length=max_length, num_beams=5, early_stopping=True)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 4: Full Pipeline - Combine SciBERT and T5
def qa_pipeline(folder_path):
    # Extract text from PDFs
    pdf_texts = extract_text_from_pdfs(folder_path)

    # Initialize SciBERT QA model and T5 generator
    scibert_qa = SciBERTQA()
    t5_generator = T5ResponseGenerator()

    for pdf in pdf_texts:
        print(f"\n--- Processing PDF: {pdf['filename']} ---\n")
        
        # Define your question (this can be dynamic or user-input)
        question = "What is the growth temperature for carbon nanotubes?"

        # Step 1: Use SciBERT to get a concise answer
        answer = scibert_qa.answer_question(question, pdf['text'])
        print(f"SciBERT Answer: {answer}")

        # Step 2: Use T5 to generate a more detailed response
        detailed_response = t5_generator.generate_response(question, answer)
        print(f"T5 Generated Response: {detailed_response}\n")

# Run the pipeline
folder_path = "./Data"  # Replace with the actual folder path containing your PDFs
qa_pipeline(folder_path)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized f

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
